# Mining words from Wikipedia

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import array_contains
import unicodedata
import pycountry
import string
from operator import add

spark = SparkSession \
    .builder \
    .appName("Analysing Wikipedia") \
    .getOrCreate()

In [3]:
df = spark.read.json("./nowiki-20210111-cirrussearch-content.json")

## Exploring the dataset

Looking at the schema just to explore the dataset. Found [a description of the JSON dump format on Wikipedia](https://meta.wikimedia.org/wiki/Data_dumps/Misc_dumps_format)

In [27]:
df.printSchema()

root
 |-- auxiliary_text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- content_model: string (nullable = true)
 |-- coordinates: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- coord: struct (nullable = true)
 |    |    |    |-- lat: double (nullable = true)
 |    |    |    |-- lon: double (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- dim: long (nullable = true)
 |    |    |-- globe: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- primary: boolean (nullable = true)
 |    |    |-- region: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- create_timestamp: string (nullable = true)
 |-- defaultsort: string (nullable = true)
 |-- display_title: string (nullable = true)
 |-- external_link: array (nullable = true)
 |    |-- element: strin

In [5]:
df.show()

+--------------------+--------------------+-------------+--------------------+--------------------+----------------+-------------+--------------------+--------------------+--------------+--------------+--------+---------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------+------------+------+-------------+
|      auxiliary_text|            category|content_model|         coordinates|    create_timestamp|     defaultsort|display_title|       external_link|             heading|incoming_links|         index|language|namespace|namespace_text|        opening_text|   ores_articletopic|  ores_articletopics|       outgoing_link|    popularity_score|            redirect|             score|         source_text|            template|                text|text_bytes|    

### Find columns to filter on

In [6]:
# Should one filter out null?
df.select("content_model").distinct().show()

+-------------+
|content_model|
+-------------+
|         null|
|     wikitext|
+-------------+



In [7]:
df.filter(df["content_model"].isNotNull()).count()

548219

In [8]:
df.filter(df["content_model"].isNull()).count()

548219

In [9]:
# It seems like every other row in this dataset consists of just nulls,
# so I remove those rows
df_not_null = df.filter(df["content_model"].isNotNull())

In [10]:
# After removing the nulls, there is only one language (where previously null also showed up)
df_not_null.select("language").distinct().show()

+--------+
|language|
+--------+
|      nb|
+--------+



In [11]:
# About namespaces https://en.wikipedia.org/wiki/Wikipedia:Namespace
# Articles have no namespace (no prefix), we are interested namespace 0.
# It seems this dataset only contains the namespace we are interested in!
df_not_null.select("namespace").distinct().show()

+---------+
|namespace|
+---------+
|        0|
+---------+



## Cleaning dataset

In [28]:
# Filtering out columns

filtered_df = df_not_null.drop("content_model", "language", "category", "coordinates", "defaultsort", \
        "external_link", "heading", "incoming_links", "namespace", "namespace_text", \
        "outgoing_link", "redirect", "text_bytes", "template", "wiki", \
        "wikibase_item", "version_type", "file_bits", "file_height", "file_media_type", \
        "file_resolution", "file_size", "file_text", "file_width", "index", \
        "file_mime", "ores_articletopic", "ores_articletopics", "score", "popularity_score", \
        "display_title", "auxiliary_text", "create_timestamp", "opening_text", "source_text", \
        "timestamp", "version")

In [13]:
# Seems like only one entry per article. That's good! I feared it was one per revision.
filtered_df.filter(filtered_df["title"] == "Kill Buljo").show()

+--------------------+----------+
|                text|     title|
+--------------------+----------+
|Kill Buljo er en ...|Kill Buljo|
+--------------------+----------+



In [14]:
# Comparing the content in the text column to the content on the web page
# https://no.wikipedia.org/wiki/Kill_Buljo
filtered_df.filter(filtered_df["title"] == "Kill Buljo").select("text").collect()[0][0]

'Kill Buljo er en norsk film fra 2008. Den er en komisk parodi på det amerikanske actioneeventyret Kill Bill, og handlingen er lagt til Finnmark. Regi er ved Tommy Wirkola og manus Stig Frode Henriksen og Tommy Wirkola. Filmen hadde premiere i mars 2007, og ble sett av 87 000 på kino i Norge. Det ble solgt over 95 000 DVD-er. Filmmusikken ble bl.a. laget av Alta-bandet Cyaneed. Kill Buljo 2 kom ut i 2013. Jompa Tormann er rollefiguren som spilles av Stig Frode Henriksen i filmen Kill Buljo. Rollefiguren dukker også opp i flere kortfilmer på DVD-en Kill Buljo: The Beginning. Jompa Tormann: Stig Frode Henriksen Pappa Buljo: Frank Arne Olsen Tampa Buljo: Martin Hykkerud Sid Wisløff: Tommy Wirkola Unni Formen: Natasha Angel Dahle Peggy Mathilassi: Linda Øverlie Nilsen Crazy Beibifeit: Ørjan Gamst Kjell Driver: John Even Pedersen Bud Light: Christian Reiertsen Lara Kofta: Merete Nordahl Mr. Handjagi: Ørjan Gamst Kato: Jørn Tore Nilsen Blow Job: Heidi Monsen Troll Tove: Eirik Junge Eliassen 

It seems the text column contains everything in the article, so I went back and removed all columns except `text` and `title`.

The text content itself could need some cleaning. There are several instances of `(en)` which looking at the web page seems to be the language of the external reference, so at least remove `(en)` and `(no)`. At one point we see `[død lenke]` which is an inline footnote, possibly a generic one. Going to the web page we find that it was created by a bot. On Wikipedia we find a web page listing [all referance template tags that can appear in running text](https://no.wikipedia.org/wiki/Mal:Trenger_referanse).

Also found a [list over sentences called "stub"](https://no.wikipedia.org/wiki/Kategori:Stubbmaler) that can appear in the running text. Unfortunately there were 361 of them, and much more template [on the list of all tempaltes](https://no.wikipedia.org/wiki/Kategori:Maler). I tried to use the other dataset (`...general.json`) to find the text the templates produces so I could remove that, but gave up.

We see that \xa0 appears.

And lastly we want to remove punctuation marks, digits and other special characters. Basically be left with only words consisting of letters.

### Cleaning the text content

In [15]:
possible_template_reference_tags = ["[trenger referanse]", "[klargjør]", "[hvor?]", "[trenger oppdatering]", "[død lenke]", "[trenger sitat]", "[trenger bedre kilde]", "[bør utdypes]", "[hvem?]", "[omstridt – diskuter]", "[ufullstendig referanse]", "[ikke i angitt kilde]", "[tredjepartskilde trengs]", "[når?]", "[av hvem?]", "[sic]", "[trenger sidetall]"]

# First time I understand this is a generator and not just a for loop
# Weirdly enough, the ISO 639-3 alpha_2 codes doesn't contain "en"
ext_ref_language_tags = [f"({country.alpha_2.lower()})" for country in pycountry.countries] + ["(en)"]

In [29]:
all_possible_template_text = []
# It is empty because I don't know! I tried to find the templates in 
# "Mining meta information about Wikipedia.ipynb", but failed

In [17]:
# Use RDD features to map and reduce
# Transform from Row with title and text to just text strings, rigth away
rdd = filtered_df.rdd.map(lambda row: row.text)

In [18]:
def remove_strings(src_string: str, strings_to_remove: list):
    new_string= src_string
    for s in strings_to_remove:
        new_string = new_string.replace(s, "")
    return new_string

In [19]:
# Remove possible_template_reference_tags
removed_templates_rdd = rdd.map(lambda s: remove_strings(s, possible_template_reference_tags))
removed_templates_rdd.take(2)

['Hundene i Riga er en svensk film fra 1995 av Per Berglund med Rolf Lassgård, Björn Kjellman, Charlotte Sieling og Paul Butkevich i noen av rollene. Filmen basert seg på romanen Hundene i Riga av Henning Mankell som er den andre i serien om Kurt Wallander. En livbåt flyter i land ved den skånske kysten. I båten befinner det seg to menn som har blitt myrdet. Etterforsker Kurt Wallander fra politiet i Ystad tilkalles til plassen. Ved hjelp av politiet i Latvia blir begge mennene identifisert. For å lette utredningen ble en politimann tilkalt fra Latvia for å hjelpe til å løse saken. Men når politimannen vender tilbake til Latvia blir han myrdet. Kurt Wallander flyr til Latvia for å forsøke å finne ut hvorfor politimannen ble myrdet. (en) Hundene i Riga på Internet Movie Database (sv) Hundene i Riga i Svensk Filmdatabas (en) Hundene i Riga på Rotten Tomatoes Portal: Film',
 'Kill Buljo er en norsk film fra 2008. Den er en komisk parodi på det amerikanske actioneeventyret Kill Bill, og ha

In [20]:
# Remove language tags
removed_lang_tags_rdd = removed_templates_rdd.map(lambda s: remove_strings(s, ext_ref_language_tags))
removed_lang_tags_rdd.take(2)

['Hundene i Riga er en svensk film fra 1995 av Per Berglund med Rolf Lassgård, Björn Kjellman, Charlotte Sieling og Paul Butkevich i noen av rollene. Filmen basert seg på romanen Hundene i Riga av Henning Mankell som er den andre i serien om Kurt Wallander. En livbåt flyter i land ved den skånske kysten. I båten befinner det seg to menn som har blitt myrdet. Etterforsker Kurt Wallander fra politiet i Ystad tilkalles til plassen. Ved hjelp av politiet i Latvia blir begge mennene identifisert. For å lette utredningen ble en politimann tilkalt fra Latvia for å hjelpe til å løse saken. Men når politimannen vender tilbake til Latvia blir han myrdet. Kurt Wallander flyr til Latvia for å forsøke å finne ut hvorfor politimannen ble myrdet.  Hundene i Riga på Internet Movie Database  Hundene i Riga i Svensk Filmdatabas  Hundene i Riga på Rotten Tomatoes Portal: Film',
 'Kill Buljo er en norsk film fra 2008. Den er en komisk parodi på det amerikanske actioneeventyret Kill Bill, og handlingen er 

In [21]:
def remove_non_letter_words_and_chars(src_string: str):
    whitelist = set('abcdefghijklmnopqrstuvwxyzæøå ABCDEFGHIJKLMNOPQRSTUVWXYZÆØÅ')
    new_string = ""
    for char in src_string:
        if char in whitelist:
            new_string = new_string + char
        else:
            new_string = new_string + " "
    return new_string

In [22]:
only_letters_rdd = removed_lang_tags_rdd.map(lambda s: remove_non_letter_words_and_chars(s))
only_letters_rdd.take(2)

['Hundene i Riga er en svensk film fra      av Per Berglund med Rolf Lassgård  Bj rn Kjellman  Charlotte Sieling og Paul Butkevich i noen av rollene  Filmen basert seg på romanen Hundene i Riga av Henning Mankell som er den andre i serien om Kurt Wallander  En livbåt flyter i land ved den skånske kysten  I båten befinner det seg to menn som har blitt myrdet  Etterforsker Kurt Wallander fra politiet i Ystad tilkalles til plassen  Ved hjelp av politiet i Latvia blir begge mennene identifisert  For å lette utredningen ble en politimann tilkalt fra Latvia for å hjelpe til å løse saken  Men når politimannen vender tilbake til Latvia blir han myrdet  Kurt Wallander flyr til Latvia for å forsøke å finne ut hvorfor politimannen ble myrdet   Hundene i Riga på Internet Movie Database  Hundene i Riga i Svensk Filmdatabas  Hundene i Riga på Rotten Tomatoes Portal  Film',
 'Kill Buljo er en norsk film fra       Den er en komisk parodi på det amerikanske actioneeventyret Kill Bill  og handlingen er 

### Pre-processing words

In [49]:
def convert_to_lowercase_if_not_acronym(src_string: str):
    for letter in src_string:
        if letter.islower():
            return src_string.lower()
    return src_string

In [50]:
%%time
# Convert string to list of lowercase words larger with two or more characters
words_rdd = only_letters_rdd \
    .flatMap(lambda s: s.split(" ")) \
    .filter(lambda word: len(word) > 1) \
    .map(lambda word: convert_to_lowercase_if_not_acronym(word))
words_rdd.take(20)

Wall time: 801 ms


['hundene',
 'riga',
 'er',
 'en',
 'svensk',
 'film',
 'fra',
 'av',
 'per',
 'berglund',
 'med',
 'rolf',
 'lassgård',
 'bj',
 'rn',
 'kjellman',
 'charlotte',
 'sieling',
 'og',
 'paul']

In [51]:
# List of stopwords taken from http://snowball.tartarus.org/algorithms/norwegian/stop.txt
stopwords = []
with open("./norwegian_stopwords.txt", "r", encoding="utf-8") as f:
    for line in f:
        strings = line.split("|")
        potential_stopword = strings[0].strip()
        if (not potential_stopword == ""):
            stopwords.append(potential_stopword)

# Remove English stopwords as well since I noticed "the", "in", and "of" in the top 20
# Inspired by the Google History stopword list found at https://www.ranks.nl/stopwords
english_stopwords = ["I", "a", "an", "are", "as", "at", "by", "com", "for", "from", "how", "in", "it", "of", "on", "that", "the", "this", "was", "what", "when", "where", "who", "will", "with", "www"]
stopwords = stopwords + english_stopwords
stopwords[:10]

['og', 'i', 'jeg', 'det', 'at', 'en', 'et', 'den', 'til', 'er']

In [52]:
def remove_word_in_list(src_word: str, list_of_words: list):
    if (src_word in list_of_words):
        return ""
    else:
        return src_word

In [53]:
# Remove all stop-words
no_stopwords_rdd = words_rdd \
    .map(lambda word: remove_word_in_list(word, stopwords)) \
    .filter(lambda word: word != "")
no_stopwords_rdd.take(20)

['hundene',
 'riga',
 'svensk',
 'film',
 'per',
 'berglund',
 'rolf',
 'lassgård',
 'bj',
 'rn',
 'kjellman',
 'charlotte',
 'sieling',
 'paul',
 'butkevich',
 'rollene',
 'filmen',
 'basert',
 'romanen',
 'hundene']

## Mining!

In [54]:
%%time
# Finding the most popular words

ranked_words_rdd = no_stopwords_rdd.map(lambda word: (word, 1)) \
    .reduceByKey(add) \
    .sortBy(lambda x: x[1], ascending=False)

endTime = time.time()
ranked_words_rdd.take(20)

Wall time: 7min 58s


[('besøkt', 535892),
 ('under', 300222),
 ('to', 278927),
 ('andre', 228236),
 ('hos', 225040),
 ('første', 218815),
 ('norsk', 218534),
 ('født', 212220),
 ('and', 211479),
 ('oktober', 197398),
 ('arkivert', 188757),
 ('år', 186704),
 ('mer', 184454),
 ('fikk', 162323),
 ('flere', 156041),
 ('januar', 154884),
 ('finnes', 151502),
 ('august', 150130),
 ('mars', 148735),
 ('OL', 144410)]

I purposefully didn't [stem](https://en.wikipedia.org/wiki/Stemming) the words, since that could lead to non-existing words. E.g. `opparbeide` -> the stem `opparbeid`, `adresse` -> `adress`. In the above RDD we find different [inflection](https://en.wikipedia.org/wiki/Inflection) of the word "artikkel", "artikkelene", "artikler", and although I don't want to stem the words, I still want to avoid having different inflections of the same root word.

After some Googling it seems what I am looking for is the [lexeme](https://en.wikipedia.org/wiki/Lemma_(morphology)), and finding it is called lemmatisation.

One solution that came to mind is to use word embeddings that contains relations between words, and then remove words that are very similar. Hopefully this would mean different inflections of "artikkel" could be removed.

### Remove grammatical bendings

In [28]:
# final_words_list = ranked_words_rdd.map(lambda t: t[0]).take(1000)
# with open("./out/wikipedia_words.txt", "w", encoding="utf-8") as f:
#     for word in final_words_list:
#         f.write(f"{word}\n")